In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import *

In [3]:
df = spark.read.csv("sentiment_data", header = True, inferSchema = True)
df.printSchema()

root
 |-- Text: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [4]:
df.count()

8272

In [5]:
df = df.na.drop()
df = df.na.replace(-1, 0)
df = df.withColumn("Sentiment", df.Sentiment.cast('double'))
df.printSchema()

root
 |-- Text: string (nullable = true)
 |-- Sentiment: double (nullable = true)



In [6]:
df.show()

+--------------------+---------+
|                Text|Sentiment|
+--------------------+---------+
|BTC should be in ...|      1.0|
|No one has convin...|     -1.0|
|I think it can be...|      1.0|
|The value of bitc...|     -1.0|
|I couldn't imagin...|     -1.0|
|Not gonna trust t...|     -1.0|
|Bitcoin is Hope &...|      1.0|
|We can no longer ...|      1.0|
|In short, invest ...|      1.0|
|I do not feel sor...|     -1.0|
|Bitcoin is pretty...|      1.0|
|I don’t even know...|     -1.0|
|"Funny how people...|     -1.0|
|If you lose money...|     -1.0|
|Yaaas 🙏🏼 buy mo...|      1.0|
|good time to accu...|      1.0|
|Dogecoin is plain...|      1.0|
|"You don't unders...|     null|
|      Not to mention|     null|
|Maybe one day it ...|     -1.0|
+--------------------+---------+
only showing top 20 rows



In [7]:
train, test = df.randomSplit([0.7, 0.3], seed=42)

In [8]:
tokenizer = Tokenizer(inputCol="Text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001, labelCol='Sentiment')
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

model = pipeline.fit(train)

22/12/30 17:59:46 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 6.0 (TID 5) (bda-hdfs-dataproc-new-w-1.europe-north1-b.c.crypto-busting.internal executor 2): scala.MatchError: [null,1.0,(262144,[27576,33180,73241,107107,196839,249180],[1.0,1.0,1.0,1.0,1.0,4.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstances$1(Predictor.scala:81)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.foldLeft(Iterator.scala:1431)
	at scala.collection.TraversableOnce.aggregate(TraversableOnce.scala:260)
	at scala.col

Py4JJavaError: An error occurred while calling o93.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 4 times, most recent failure: Lost task 0.3 in stage 6.0 (TID 8) (bda-hdfs-dataproc-new-w-0.europe-north1-b.c.crypto-busting.internal executor 1): scala.MatchError: [null,1.0,(262144,[27576,33180,73241,107107,196839,249180],[1.0,1.0,1.0,1.0,1.0,4.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstances$1(Predictor.scala:81)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.f

In [ ]:
# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
prediction.printSchema()

In [ ]:
eval = BinaryClassificationEvaluator(labelCol="Sentiment")
eval.evaluate(prediction)

In [ ]:
# hist_reddit_schema = StructType(
#    [StructField('subreddit', StringType(), True),
#     StructField('title', StringType(), True),
#     StructField('selftext', StringType(), True),
#     StructField('created_utc', TimestampType(), True),
#    ]
#   )

In [ ]:
# df_red = spark.read.csv("historical_reddit", header = True, inferSchema = True)
# df_red.printSchema()

In [ ]:
# df_red.show()

In [ ]:
# Prosty model regresji na szeregów czasowych najlpiej na historical 
# Testy na historical dla MSE
# Predykcja powinna opierac się na cenach z ostanich 24h + na obecnym / przeszłym sentymencie z reddita.
# Na razie tylko na podstawie crypto* na nast. godzinę
# Zapisywać i wczytać te modele jakoś?

In [ ]:
df_cryp = spark.read.csv("historical_crypto", header = True, inferSchema = True)
df_cryp.printSchema()